# Documentation assistant

This notebook demonstrates a documentation assistant: Video-to-documentation conversion using Google GenAi

Converting videos-to-documentation involves following steps: 

## Protocol finder:
1. Video_analyzer_agent: Analyzes video to get the laboratory procedure that can be compared with protocols on Confluence.
2. Protocol_agent: Select protocol name which best captures the actions being performed in the video

## Lab note generator:
3. Video comparing to ground-of-truth protocol → lab documentation + errors in procedure
4. Analytics based on benchmark dataset: automatic comparison of errors found by documentation assistent vs actual errors


In [ ]:
from __future__ import annotations

%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path

path_to_append = Path(Path.cwd()).parent
sys.path.append(str(path_to_append))
from proteomics_specialist.sub_agents.video_analyzer_agent import agent

In [ ]:
# Step 1: Analyzing video to get the laboratory procedure

test_cases = [
    # 'Video path: /Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/DisconnectingColumn_docuCorrect.MP4. Analyze the video.',
    # "Analyse this video: \"/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/documentation/DisconnectingColumn_docuCorrect.MP4\".",
    # 'Analyse this video: "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4". What is happening?',
    # 'Analyse this video: "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4".',
    "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4",
    # 'Analyse this video: "/Users/patriciaskowronek/Downloads/ultra_short.mp4".. What is happening?',
    # '""/Users/patriciaskowronek/Downloads/ultra_short.mp4"."',
    # '"/Users/patriciaskowronek/Downloads/ultra_short.mp4".',
    # "Check this document.pdf for errors.",
    # "Analyse this video: . what is happening.",
]

for test in test_cases:
    result = agent.analyze_proteomics_video(test)
    print(result)
    print()

In [ ]:
from IPython.display import Markdown

from proteomics_specialist.sub_agents.lab_note_generator_agent import agent

result = agent.generate_lab_notes(
    query="gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4",
    tool_context=None,
    protocol_input="""
    # Connecting IonOpticks Column & sample line of Evosep

    ## Abstract
    This protocol explains how to connect an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.


    ## Materials

    ### Equipment
    - IonOpticks Column
    - timsTOF Ultra Mass Spectrometer
      - Equipped with UltraSource ion source
      - Equipped with column oven for temperature control
    - Evosep One LC System
      - with sample line
    - NanoViper Adapter (black)
    - Pliers


    ## Procedure
    *Estimated timing: 3 minutes*

    1. Verfy that the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standing by mode.

    ### Connect column and sample line
    2. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.
    3. Attach a black NanoViper adapter to the LC sample line.
    4. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.
    5. Hold the column fititng of the IonOpticks column with a pliers.
    6. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 1).
      CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.
    7. Remove the NanoViper adapter so that the oven can be tightly closed.
    8. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).
    9. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.
    10. Close the lid of the oven.
    11. Verify the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven, respectively.

    ### Switch timsTOF to operate and idle flow
    12. In timsControl, transition the instrument to the operation mode by using the on/off button.
    13. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.
    14. Return to timsControl. Check the MS signal. It should be around 2.5x10^7.


    ## Expected Results
    - In timsControl, signal intensity should be above 10^7
    - Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)


    ## Figures
    ### Figure 1: Connection of a sample line of a LC system with a separation column and emitter.**

    ### Figure 2: UltraSource.**


    ## References
    1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation-serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w
    """,
)
Markdown(result["lab_notes"])

In [ ]:
# Uploading a single file to gcs
import os

from google.cloud import storage

from proteomics_specialist.sub_agents import utils

bucket_name = os.getenv("GOOGLE_CLOUD_STORAGE_BUCKET")
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)

path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/Dilute_docuWrongVolume_PipettTipNotChanged.md"
file_path, file_uri, filename = utils.upload_file_from_path_to_gcs(
    path, bucket, "lab_note_generation_examples"
)
print(file_path, file_uri, filename)

In [ ]:
# Loading the content of a md file
from IPython.display import Markdown


def load_md_as_text(file_path: str) -> None:
    """Load markdown file as plain text string."""
    with Path.open(file_path, encoding="utf-8") as file:
        return file.read()


md_content = load_md_as_text(
    "/Users/patriciaskowronek/Documents/proteomics_specialist/data/Dilute_docuWrongVolume_PipettTipNotChanged.md"
)
md_content

In [ ]:
# Generting the benchmark_data.csv
# ruff: noqa: RUF001

import pandas as pd

benchmark_dataset_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/benchmark_data.csv"

df_benchmark_data = pd.read_csv(benchmark_dataset_path, index_col=0)
new_row_dict1 = {
    "eval_set_name": "PlaceEvotips_docuCorrect",
    "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
    "video_path": "gs://ai-proteomics-advisor/lab_note_generation_examples/PlaceEvotips_docuCorrect.MP4",
    "recording_type": "camera",
    "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. The Evotip box intended for S1 was verified to be filled with Buffer A solution by visual inspection. Verification was omitted for the box placed at S3 since it will only contain blanks.\n2. Placed Evotip Box at S1 within the rack system of the Evosep instrument. Ensured box is firmly seated in its designated position.\n3. Placed an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. Inspected each Evotip before placement to verify its condition. All Evotips were properly prepared as they display a pale-colored SPE material disc with visible solvent above it.\n5. Placed the verified Evotips into the prepared Evotip boxes at S1 from A1 to A12.\n6. Placed empty Evotips, called Blanks, at S3 from A1 to A12.\n7. ❌ Omitted: Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Evotips that are placed at S1 from A1 to A6.\n- Blanks placed at S3 from A1 to A6.\n",
    "error_dict": """[{"Step": 1.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 2.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 3.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 4.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 5.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 6.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 7.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}]""",
    "comments": "",
}
new_row_dict2 = {
    "eval_set_name": "PlaceEvotips_docuWrongPosition",
    "protocol": """The page titled "Placing Evotips in Evotip Boxes on the Evosep One System" contains following information: - ## Abstract\nThis protocol describes the proper procedure for inspecting Evotips and placing Evotips in Evotip boxes on the liquid chromatography system Evosep One.\n\n\n## Materials\n\n### Equipment\n- Evotips\n  - Single-use stage tips for sample injection\n  - Rack layout: Two columns (left and right)\n  - Left column (top to bottom): S1, S2, S3\n  - Right column (top to bottom): S4, S5, S6\n  - Within each box: Standard 96-well format with A1 (top left), A12 (top right), H12 (bottom right)\n- Evotip Boxes\n  - 96-well format (A1-H12) (Figure 1)\n- Evosep One System\n  - Liquid chromatography system\n\n### Reagents\n- Formic acid (FA)\n  ! CAUTION: This liquid may be corrosive. It is harmful and can cause damage if direct contact occurs.\n\n### Reagent setup\n- Buffer A: Consists of 0.1% (vol/vol) FA. The buffers are stable for at least 6 months at room temperature as long as they are protected from sunlight.\n\n\n## Procedure\n*Estimated timing: less than 1 minute*\n\n1. Visually inspect the Evotip box intended for S1 to verify it's filled with Buffer A solution to a minimum depth of 1 cm. Verification of the Evotip box placed at S3 is not necessary as it will only contain blanks.\n2. Place Evotip Box at S1 within the rack system of the Evosep instrument. Ensure each box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensure each box is firmly seated in its designated position.\n4. Inspect each Evotip before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it (Figure 2).\n   **CRITICAL STEP**: Discard any Evotips showing signs of dryness or displaying a white-colored disc, as these conditions indicate compromised functionality that could affect sample analysis.\n5. Place the verified Evotips into the prepared Evotip boxes at S1 from A1 to A6.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A6.\n7. Document the precise position of each placed Evotip.\n\n\n## Expected Results\nWhen the procedure is performed correctly, you should observe:\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Accurate documentation of tip positions: Evotips that are placed at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Figures\n\n### Figure 1: Evosep positions\n[Placeholder for close-up of single Evotip box showing well positions (A1-H12)]\n\n### Figure 2: Evotip Quality Assessment\n- Most Evotips: Properly hydrated Evotip with pale-colored disc and visible solvent\n- Orange-highlighted Evotip: Compromised Evotip showing white/dry disc\n\n\n## References\n1. Evosep One - User Guide: https://www.evosep.com/wp-content/uploads/2024/06/Evosep-One-User-Guide-v18.pdf\n2. Sample loading protocol for Evotips: https://www.evosep.com/wp-content/uploads/2020/03/Sample-loading-protocol.pdf\n""",
    "video_path": "gs://ai-proteomics-advisor/lab_note_generation_examples/PlaceEvotips_docuWrongPosition.MP4",
    "recording_type": "camera",
    "ground_truth_lab_notes": "# Placing Evotips in Evotip Boxes on the Evosep One System\n\n## Aim\nPlacing Evotips in Evotip boxes: Evotips with HeLa at S1 from A1 to A6 and blanks placed at S3 from A1 to A6.\n\n\n## Materials\n\n### Equipment\n- Evotips\n- Evotip Boxes\n- Evosep One System\n\n### Reagent setup\n- Buffer A:0.1% (vol/vol) FA\n\n\n## Procedure\n*Timing: less than 1 minute*\n\n1. The Evotip box intended for S1 was verified to be filled with Buffer A solution by visual inspection. Verification was omitted for the box placed at S3 since it will only contain blanks.\n2. Placed Evotip Box at S1 within the rack system of the Evosep instrument. Ensured box is firmly seated in its designated position.\n3. Place an empty Evotip Box for Blank tips at S3. Ensured box is firmly seated in its designated position.\n4. ❌ **Omitted:** Inspected only some of the Evotips before placement to verify its condition. Properly prepared Evotips should display a pale-colored SPE material disc with visible solvent above it. All Evotips were fine.\n5. ❌ **Error:** Placed the verified Evotips into the prepared Evotip boxes at S1, but positioned them from B1 to B3 and B5 to B7.\n6. Place empty Evotips, called Blanks, at S3 from A1 to A12.\n7. ❌ **Omitted:** Did not document the precise position of each placed Evotip during the recorded procedure.\n\n\n## Results\n- Properly seated Evotip boxes in the rack system\n- Visible Buffer A solution in boxes (1 cm depth)\n- All non-blank Evotips showing pale-colored SPE material discs & clear solvent meniscus above each SPE disc of each Evotip\n- Evotips that are placed at S1 from B1 to B3 and B5 to B7.\n- Blanks placed at S3 from A1 to A6.\n",
    "error_dict": """[{"Step": 1.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 2.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 3.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 4.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "Fast"},{"Step": 5.0, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialOrientation"},{"Step": 6.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 7.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "GeneralKnowledge"}]""",
    "comments": "Step 4: Evotip placed at wrong position",
}
new_row_dict3 = {
    "eval_set_name": "ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl",
    "protocol": """The page titled ""Connecting IonOpticks column & sample line"" contains following information: - ## Abstract\nThis protocol explains how to connect an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Ultra Mass Spectrometer\n  - Equipped with UltraSource ion source\n  - Equipped with column oven for temperature control\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Estimated timing: 3 minutes*\n\n1. Verfy that the instrument is in standing by mode. If not press the on/off button to switch from operate mode to standing by mode.\n\n### Connect column and sample line\n2. Verify that an IonOpticks column is already inserted into the UltraSource of the timsTOF Ultra.\n3. Attach a black NanoViper adapter to the LC sample line.\n4. Remove any access liquid at the top of the nanoViper of the sample line for instance by snipping it off.\n5. Hold the column fititng of the IonOpticks column with a pliers.\n6. Hand-tighten the NanoViper of the LC sample line with the column fitting (Figure 1).\n   CRITICAL STEP: Be careful not to overtighten the connection between the IonOpticks column and the timsTOF Ultra. Otherwise you can damage the column and the LC sample line.\n7. Remove the NanoViper adapter so that the oven can be tightly closed.\n8. Draw the oven closer to the UltraSource, secure it with the screw on the bottom of the oven (Figure 2).\n9. Lift and place the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n10. Close the lid of the oven.\n11. Verify the temperature at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven, respectively.\n\n### Switch timsTOF to operate and idle flow\n12. In timsControl, transition the instrument to the operation mode by using the on/off button.\n13. Navigate to Hystar and ensure that the idle flow is active. If not right-click on the Evosep logo, choosing 'preparation', and then selecting 'idle flow' and 'Run'.\n14. Return to timsControl. Check the MS signal. It should be around 2.5x10^7.\n\n\n## Expected Results\n- In timsControl, signal intensity should be above 10^7\n- Stable signal in in timsControl (in the windows for Mobilogram (timsView) and mass spectrum)\n\n\n## Figures\n### Figure 1: Connection of a sample line of a LC system with a separation column and emitter.**\n\n### Figure 2: UltraSource.**\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation-serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n""",
    "video_path": "gs://ai-proteomics-advisor/input_video/ConnectingColumnSampleLine_docuWithoutStandbyANDtimsControl.mp4",
    "recording_type": "camera",
    "ground_truth_lab_notes": """# Connecting IonOpticks Column & sample line of Evosep\n\n## Aim\nConnecting an IonOpticks column inserted into a timsTOF Ultra with an sample line of an Evosep.\n\n\n## Materials\n\n### Equipment\n- IonOpticks Column\n- timsTOF Ultra Mass Spectrometer\n  - Equipped with UltraSource ion source\n  - Equipped with column oven for temperature control\n- Evosep One LC System\n  - with sample line\n- NanoViper Adapter (black)\n- Pliers\n\n\n## Procedure\n*Timing: 3 minutes*\n\n1. ❌ **Omitted:** Did not verify that the instrument was in standing by mode.\n\n### Connected column and sample line\n2. Verified that an IonOpticks column was already inserted into the UltraSource of the timsTOF Ultra.\n3. Attached a black NanoViper adapter to the LC sample line.\n4. Removed any excess liquid at the top of the nanoViper of the sample line by snipping it off.\n5. Held the column fitting of the IonOpticks column with pliers.\n6. Hand-tightened the NanoViper of the LC sample line with the column fitting.\n7. Removed the NanoViper adapter.\n8. ❌ **Omitted:** Did not draw the oven closer to the UltraSource or secure it with the screw on the bottom of the oven.\n9. Lifted and placed the metal grounding screw at the column-sample line connection to establish proper ESI spray grounding with the column oven.\n10. Closed the lid of the oven.\n10.1 ➕ **Added:** Connected the oven to the electrical power supply.\n11. Verified the temperature was at 50°C for IonOpticks columns, as indicated by three illuminated LEDs on the column oven.\n\n### Switched timsTOF to operate and idle flow\n12. ❌ **Omitted:** Skipped to transition the instrument to the operation mode by using the on/off button in timsControl.\n13. ❌ **Omitted:** Did not navigate to Hystar and ensure that the idle flow was active.\n14. ❌ **Omitted:** Did not return to timsControl to check the MS signal.\n\n\n## Results\nLast part of the protocol is not on video. Outcome is unclear.\n""",
    "error_dict": """[{'Step': 1.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'Fast'}, {'Step': 2.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 3.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 4.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 5.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 6.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 7.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 8.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}, {'Step': 9.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 10.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 10.1, 'Benchmark': 'Error', 'Class': 'Addition', 'Skill': 'ProteomicsKnowledge'}, {'Step': 11.0, 'Benchmark': 'No Error', 'Class': 'N/A', 'Skill': 'N/A'}, {'Step': 12.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'Fast'}, {'Step': 13.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}, {'Step': 14.0, 'Benchmark': 'Error', 'Class': 'Omitted', 'Skill': 'ProteomicsKnowledge'}]""",
    "comments": "Did not verify if in standby, Added connecting oven, not on video: Switched timsTOF to operate and idle flow",
}
new_row_dict4 = {
    "eval_set_name": "QueueSamples_docuNoBlankNoSampleIDWrongMSmethod",
    "protocol": """# Queue and measure samples in HyStar\n\n\n## Abstract\nThis protocol explains how to queue sample in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\n| Software | Version | Purpose | Source/Link |\n|----------|---------|---------|------------|\n| HyStar | 6.0 | Controls LC and MS | On every instrument computer |\n\n\n## Procedure\n*Estimated timing: 6 minutes*\n\n1. Verify the Evotips were placed on top of the Evosep LC system. In this example, 5 ng HeLa Evotips were placed at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n    Note: Blank Evotis can be unused and dry Evotips.\n2. Verify that the TIMS device in TimsControl is calibrated. The TIMS device should be calibrated each time before you start a sample queue.\n3. In Hystar, navigate to the \'Acquisition\' tab.\n4. Either select \'New\', and subsequently choose \'LC-MS sample table\' (Figure 1) to generate a new sample table or choose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. In both cases, copy already existing sample table entries to modify them.\n6. Adjust the sample ID so that it follows this pattern: currentDate_massSpec_user_sampleType_projectID_ sampleName. Typical examples for sampleType: "SA_blank", "MA_HeLa", "DIAMA_HeLa".\n7. For performance evaluation of the LC-MS system, we recommend to queue one blank, three dda-PASEF runs, three dia-PASEF runs and ending with another blank.\n    Critical step: Always start the queue with a sacrificial Evotip such as a blank tip as the first run might have altered chromatographic conditions that could introduce technical errors to your measurements.\n8. Verify the column autocompletion settings with right-click on a field in the column \'vial\' such as S1-A1 in Figure 1 and select \'Configure\'. The arrows allow one to define the direction in which the vial positions on the 96-well are autocompleted when dragging values similar to Excel in the sample table. Decide whether the values should increase from A1-A12 indicated by arrows pointing to right. Ensure that the tray type is set to \'Evosep\' and slots 1-6 are designated as \'96Evotip\'. Press \'OK\'.\n9. Match the Evotip position with the sample\'s location in the Evotip box: From S1 from A1 to A6 and blanks at S3 from A1 to A6. To do this, press the arrow next to the value in the \'vial\' column (Figure 1). Select the position where the first Evotip is placed, for instance S1 A1. You can then either specify all remaining positions automatically by dragging the values (similar to Excel\'s auto-fill function) or specify each position individually.\n10. Specify a \'path\' folder for storing the raw files.\n11. Choose an existing separation method or create a new one. To create a new separation method, right-click on the separation method field, select \'new method\' followed by \'edit method\'. Choose the method type listed under \'name\' and then press \'OK\'. Specify its name and save it.\n12. Set the injection method to \'standard\'.\n13. At \'MS method\', load either dda-PASEF and dia-PASEF maintenance methods to check the LC-MS performance or the method intended to be used for measuring the study.\n14. Stop the idle flow on the Evosep by right-clicking on the Evosep logo and selecting \'Cancel maintenance procedure\'.\n15. Save the sample table.\n16. Right-click somewhere on the top row of the freshly defined sample table entries and select \'upload sample conditions\' to pre-check if all LC and MS method can be loaded correctly. The status should be change to loaded.\n17. Press \'start\' and \'start sequence\'.\n\n\n## Expected Results\nTypical chromatogram shape with MS TIC (green line) increasing drastically around 4 minutes and another increase (wash-out) at 15 minutes. Higher MS2 intensities (red line) for dia-PASEF.\n\n\n## Figures\n### Figure 1: Hystar\n[Placeholder for Screenshot of reaauired Hystar settings]\n\n\n## References\n1. Skowronek, P., Wallmann, G., Wahle, M. et al. An accessible workflow for high-sensitivity proteomics using parallel accumulation-serial fragmentation (PASEF). Nat Protoc (2025). https://doi.org/10.1038/s41596-024-01104-w\n2. TODO: Refer to Hystar manual\n3. TODO: Maybe upload video which shows protocol in action\n""",
    "video_path": "gs://ai-proteomics-advisor/lab_note_generation_examples/QueueSamples_docuNoBlankNoSampleIDWrongMSmethod.mov",
    "recording_type": "screen_recording",
    "ground_truth_lab_notes": """# Queue and measure samples in HyStar\n\n\n## Aim\nQueuing samples in HyStar for LC-MS measurement.\n\n\n## Materials\n\n### Software\nHyStar 6.0\n\n\n## Procedure\n*Timing: 2 minutes*\n\n1. Verified that Evotips are placed on top of Evosep. 5 ng HeLa Evotips at S1 from A1 to A6 and blanks at S3 from A1 to A6.\n2. Verified that the calibration of the TIMS device in TimsControl is valid.\n3. In Hystar, navigated to the 'Acquisition' tab.\n4. Chose an already existing sample table by pressing the arrow down button when hovering over the sample table name in the left sample table column.\n5. Copied already existing sample table entries to modify them.\n7. ❌ **Error:** Queued three dda-PASEF runs, three dia-PASEF runs, but missed to queue a blank at the start and end of queue as recommended.\n8. Verified the column autocompletion settings by right-clicking on a field in the column 'vial' and selected 'Configure'. Values were set to autocomplete from A1-A12 indicated by arrows pointing to right. Ensured that the tray type was set to 'Evosep' and slots 1-6 were designated as '96Evotip'. Pressed 'OK'.\n9. Matched the Evotip position with the sample's location in the Evotip box: S1 from A1 to A6 and blanks at S3 from A1 to A6.\n13. ⚠️ **Deviation: Altered step order** & ❌ **Error:** At 'MS method', set dda-PASEF maintenance method for all samples. However, the last three samples would have to be linked to a dia-PASEF maintenance method.\n6. ⚠️ **Deviation: Altered step order** & ❌ **Error:** Adjusted the sample ID so that it followed this pattern: currentDate_massSpec_user_sampleType_sampleName, but missed to include projectID.\n10. Verified 'path' folder for storing the raw files.\n11. Verified separation method.\n12. Verified that injection method is set to 'standard'.\n15. ⚠️ **Deviation: Altered step order** Saved the sample table.\n14. Stopped the idle flow on the Evosep by right-clicking on the Evosep logo and selecting 'Cancel maintenance procedure'.\n16. ❌ **Omitted:** Missed to right-click on the top row of the freshly defined sample table entries and select 'upload sample conditions' to pre-check if all LC and MS methods could be loaded correctly. The status did not changed to loaded.\n17. Pressed 'start' and 'start sequence'.\n\n\n## Results\nChromatogram is not shown in video.\n""",
    "error_dict": """[{"Step": 1.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 2.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 3.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 4.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 5.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 6.0, "Benchmark": "Error", "Class": "Deviation & Error", "Skill": "SpatialResolution"},{"Step": 7.0, "Benchmark": "Error", "Class": "Error", "Skill": "SpatialResolution"},{"Step": 8.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 9.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 10.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 11.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 12.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 13.0, "Benchmark": "Error", "Class": "Deviation & Error", "Skill": "SpatialResolution"},{"Step": 14.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"},{"Step": 15.0, "Benchmark": "Error", "Class": "Deviation", "Skill": "ProteomicsKnowledge"},{"Step": 16.0, "Benchmark": "Error", "Class": "Omitted", "Skill": "ProteomicsKnowledge"},{"Step": 17.0, "Benchmark": "No Error", "Class": "N/A", "Skill": "N/A"}]""",
    "comments": "No blanks queued, SampleID not included at sample name, Wrong MS method for last three samples, changed order, omitted upload sample conditions",
}
df_benchmark_data = pd.DataFrame(
    [new_row_dict1, new_row_dict2, new_row_dict3, new_row_dict4]
)
# df_benchmark_data = pd.concat([df_benchmark_data, pd.DataFrame([new_row_dict])], ignore_index=True)
df_benchmark_data.to_csv(benchmark_dataset_path, index=False)
df_benchmark_data

In [ ]:
# Runing lab note eval pipline
from eval.eval_lab_note_generation.eval_analysis_run import EvaluationAnalyzer

output_dir = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/eval_lab_note_results/result_20250803_061506"
analyzer = EvaluationAnalyzer(output_dir=output_dir)
metrics_dict = analyzer.run_complete_analysis(
    Path(output_dir) / "all_eval_sets_all_runs.json"
)
metrics_dict

In [ ]:
# Inspecting individual results from log file
import json

from IPython.display import Markdown


def load_json_basic(file_path: str) -> None:
    """Load JSON file using built-in json module."""
    with Path.open(file_path, encoding="utf-8") as file:
        return json.load(file)


file_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/eval/eval_lab_note_generation/eval_lab_note_results/result_20250803_061506/eval_set_PlaceEvotips_docuCorrect_all_runs.json"
json_data = load_json_basic(file_path)
Markdown(json_data[0]["lab_notes"])

In [ ]:
pd.DataFrame(json_data[0]["df_errors"])